In [12]:
import re
import pandas as pd
from konlpy.tag import Okt

market_reviews_df = pd.read_csv('total_reviews_market.csv', encoding = 'utf-8')
market_reviews_df.head()

,category,productname,content,like,datetime
0,간식·과자·떡,[밀클레버] 미니 붕어빵 골라담기 (택2),날씨가 점점 추워져 붕어빵 생각이 간절하던 차인데 아쉽게도 제가 사는 곳은 붕세권이...,4,2023.11.01
1,간식·과자·떡,[밀클레버] 미니 붕어빵 골라담기 (택2),맛있가한데 울집 아이들은 호불호가 있는 맛이에요.,0,2023.11.14
2,간식·과자·떡,[밀클레버] 미니 붕어빵 골라담기 (택2),아이들이 슈크림붕어빵에 반했어요.^^,0,2023.11.14
3,간식·과자·떡,[밀클레버] 미니 붕어빵 골라담기 (택2),에어프라이기에 예열않고 10분정도 돌렸더니 뜨근하고 맛난 미니붕어빵이 되었어요.,0,2023.11.14
4,간식·과자·떡,[밀클레버] 미니 붕어빵 골라담기 (택2),애정하는 붕어빵 사랑해용,0,2023.11.13


In [15]:
def preprocess_text(text):
    # 1. 텍스트 정규화
    text = text.lower() # 모두 소문자로 변환
    text = re.sub(r'[^\w\s]', ' ', text) # 문장 부호 및 특수 문자 제거

    # 2. 숫자+단위, 단위+ 숫자 제거
    text = re.sub(r'\d+[가-힣]+', '', text) # Remove patterns like 숫자+단위
    text = re.sub(r'[가-힣]+\d+', '', text) # Remove patterns like 단위+숫자

    # 3. 2회 이상 띄어스기 제거
    text = re.sub(r'\s+', ' ', text)
    
    # 4. Tokenization and stemming
    okt = Okt()
    tokens = okt.morphs(text)

    # 5. 불용어 제거
    stopwords = ['이', '그', '것', '의', '을', '를', '와', '과']
    filtered_tokens = [word for word in tokens if word not in stopwords]

    return ' '.join(filtered_tokens)

def preprocess_product_name(text):
    #1. 숫자+단위, 단위+ 숫자 제거
    text = re.sub(r'\d+[가-힣]+', '', text) # Remove patterns like 숫자+단위
    text = re.sub(r'[가-힣]+\d+', '', text) # Remove patterns like 단위+숫자
    
    #2. 제품 이름에서 특수 문자와 숫자 영어 제거
    text = re.sub(r'[^가-힣\s]', '', text) # 한글과 영문자만 남기고 제거
    
    #3. 구매 시 제거
    text = re.sub(r'구매 시', ' ', text)
    
    #4. 2회 이상 띄어스기 제거
    text = re.sub(r'\s+', ' ', text)
    
    return text

# CSV 파일 로드
df = pd.read_csv('C:/Users/kgw57/PycharmProjects/data_contest/preprocessed_review/total_reviews_market.csv')

# 데이터 전처리 및 새로운 열 생성
df['preprocessed_category'] = df['category'].apply(preprocess_product_name)
df['preprocessed_productname'] = df['productname'].apply(preprocess_product_name) 
df['preprocessed_content'] = df['content'].apply(preprocess_text)

# CSV 파일로 저장
df.to_csv('preprocessed_total_reviews_market.csv', index=False)